## 1. feladat

Google maps-en jelöljük ki a 29-es busz útvonalát!

Adatok: https://bkk.hu/apps/gtfs/

In [1]:
import sys
import zipfile
path_to_zip_file = "budapest_gtfs (1).zip"
directory_to_extract_to = './data'
zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [2]:
!ls -al ./data

total 488656
drwxr-xr-x  11 macair  staff        352 Sep 27 06:03 .
drwxr-xr-x  13 macair  staff        416 Oct  1 23:17 ..
-rw-r--r--   1 macair  staff        135 Oct  1 23:17 agency.txt
-rw-r--r--@  1 macair  staff     238588 Oct  1 23:17 calendar_dates.txt
-rw-r--r--   1 macair  staff        176 Oct  1 23:17 feed_info.txt
-rw-r--r--   1 macair  staff      22885 Oct  1 23:17 pathways.txt
-rw-r--r--   1 macair  staff      28056 Oct  1 23:17 routes.txt
-rw-r--r--   1 macair  staff   13220475 Oct  1 23:17 shapes.txt
-rw-r--r--   1 macair  staff  217061871 Oct  1 23:17 stop_times.txt
-rw-r--r--   1 macair  staff     326826 Oct  1 23:17 stops.txt
-rw-r--r--   1 macair  staff   19272508 Oct  1 23:17 trips.txt


In [3]:
import json
import os

for fname in os.listdir("./data"):
    filename = './data/'+fname
    with open(filename) as f:
        print(filename)
        print("Header:",f.readline().strip())
        print("Sample:",f.readline().strip())
        print("")

./data/agency.txt
('Header:', 'agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone')
('Sample:', 'BKK,BKK,http://www.bkk.hu,Europe/Budapest,hu,+36 1 3 255 255')

./data/calendar_dates.txt
('Header:', 'service_id,date,exception_type')
('Sample:', 'B05716AHPKP-0011,20180828,1')

./data/stop_times.txt
('Header:', 'trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled')
('Sample:', 'B0571610,F04181,08:03:00,08:03:00,0,,,,0.0')

./data/shapes.txt
('Header:', 'shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,shape_dist_traveled')
('Sample:', '0803,100001,47.559670,19.082379,0.0')

./data/trips.txt
('Header:', 'route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,boarding_door')
('Sample:', '8140,B0571610,B05716AHPKP-0011,"Soroks\xc3\xa1r, Moln\xc3\xa1r-sziget",0,B05716AHPKP-0011_1,Y701,1,1,2')

./data/feed_info.txt
('Header:', 'feed_publisher_name,

### Beolvasás

A csv modul DictReader-ével tudunk soronként végigmenni a csv fájlokon, és dictionary-ként kezelni az egyes sorokat.

Gábor rájött, hogy a 29-es busz adatait a routes.txt-ben kell keresnünk:

In [4]:
import csv
import json
with open('./data/routes.txt') as f:
    csvreader = csv.DictReader(f, delimiter=',', quotechar='"')
    for row in csvreader:
        if row['route_short_name'] == '29':
            print(json.dumps(row, indent=True))

{
 "route_long_name": "", 
 "route_type": "3", 
 "route_text_color": "FFFFFF", 
 "agency_id": "BKK", 
 "route_id": "0290", 
 "route_color": "009FE3", 
 "route_desc": "Szentl\u00e9lek t\u00e9r H / H\u0171v\u00f6sv\u00f6lgy", 
 "route_short_name": "29"
}


Csináljunk rá egy függvényt, és hívjuk meg! Ha mindent jól csináltunk, ugyanaz lesz a végeredmény, mint feljebb:

In [5]:
def get_data_from(filename, column, value):
    with open(filename) as f:
        csvreader = csv.DictReader(f, delimiter=',', quotechar='"')
        for row in csvreader:
            if row[column] == value:
                return row

route = get_data_from('./data/routes.txt', 'route_short_name', '29')
route

{'agency_id': 'BKK',
 'route_color': '009FE3',
 'route_desc': 'Szentl\xc3\xa9lek t\xc3\xa9r H / H\xc5\xb1v\xc3\xb6sv\xc3\xb6lgy',
 'route_id': '0290',
 'route_long_name': '',
 'route_short_name': '29',
 'route_text_color': 'FFFFFF',
 'route_type': '3'}

Akár több adatot is kinyerhetünk, ha tudjuk, hogy mi a közös bennük:

In [6]:
def get_list_from(filename, column, value):
    result = []
    with open(filename) as f:
        csvreader = csv.DictReader(f, delimiter=',', quotechar='"')
        for row in csvreader:
            if row[column] == value:
                result.append(row)
    return result

András rájött, hogy a következő fájl, amiből adatokat kéne olvasnunk, az a trips.txt, mivel a route_id-t ő is tartalmazza, így valószínűleg van összeköttetés a két fájl között a route_id formájában. A fenti eredményünkből kiderül, hogy 29-es buszhoz tartozó route_id az a "0290"

In [7]:
trips = get_list_from('./data/trips.txt', 'route_id', route['route_id'])
trips[0:3]

[{'bikes_allowed': '2',
  'block_id': 'B74501ASZLB-0021_2',
  'boarding_door': '1',
  'direction_id': '1',
  'route_id': '0290',
  'service_id': 'B74501ASZLB-0021',
  'shape_id': 'U006',
  'trip_headsign': 'Szentl\xc3\xa9lek t\xc3\xa9r H',
  'trip_id': 'B7450110',
  'wheelchair_accessible': '1'},
 {'bikes_allowed': '2',
  'block_id': 'B74501ASZLB-0021_4',
  'boarding_door': '1',
  'direction_id': '1',
  'route_id': '0290',
  'service_id': 'B74501ASZLB-0021',
  'shape_id': 'U006',
  'trip_headsign': 'Szentl\xc3\xa9lek t\xc3\xa9r H',
  'trip_id': 'B74501100',
  'wheelchair_accessible': '1'},
 {'bikes_allowed': '2',
  'block_id': 'B74501ASZLB-0021_5',
  'boarding_door': '1',
  'direction_id': '0',
  'route_id': '0290',
  'service_id': 'B74501ASZLB-0021',
  'shape_id': 'U277',
  'trip_headsign': 'H\xc5\xb1v\xc3\xb6sv\xc3\xb6lgy',
  'trip_id': 'B74501101',
  'wheelchair_accessible': '1'}]

András rájött arra is, hogy a trip_id nevű oszlop létezik a stop_times.txt-ben is, valamint, hogy a stop_times.txt és a stops.txt összeköttetésben van a stop_id által. Így már könnyű lesz kiolvasni a stops.txt-ből a stop_lat és stop_lon értékeket egy adott megállóhoz, és kirajzolni a térképre:

In [8]:
route = get_data_from('./data/routes.txt', 'route_short_name', '29')
#ide jöhet a többi kód

### Output

Ilyen formában kéne az adatoknak generálódniuk:

In [9]:
bus_stops = [
    {'location': (47.542635, 18.964753)},
    {'location': (47.536127, 18.970568)},
    {'location': (47.534574, 18.973997)},
    {'location': (47.530852, 18.978252)},
    {'location': (47.528371, 18.980938)},
    {'location': (47.525837, 18.984168)},
    {'location': (47.52226, 18.990825)},
    {'location': (47.523548, 18.993187)},
    {'location': (47.528352, 18.996317)},
    {'location': (47.528471, 18.999995)},
    {'location': (47.529117, 19.003624)},
    {'location': (47.529779, 19.009078)},
    {'location': (47.529393, 19.011163)},
    {'location': (47.529311, 19.017049)},
    {'location': (47.52855, 19.021629)},
    {'location': (47.528721, 19.024165)},
    {'location': (47.528227, 19.0271)},
    {'location': (47.527562, 19.028415)},
    {'location': (47.527131, 19.031417)},
    {'location': (47.528039, 19.033422)},
    {'location': (47.52759, 19.036729)},
    {'location': (47.528586, 19.039001)},
    {'location': (47.531833, 19.040023)},
    {'location': (47.535152, 19.039969)},
    {'location': (47.538723, 19.0405)},
    {'location': (47.539919, 19.042014)},
    {'location': (47.539095, 19.045373)},
    {'location': (47.539949, 19.046781)}
]

## Ábrázolás a térképen

gmaps nem működött megbízhatóan, 
vmi más megoldást kellett keresni:

https://ipyleaflet.readthedocs.io/en/latest/

Telepítés:

```
pip install ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet
```

In [10]:
from ipyleaflet import Marker
from ipyleaflet import Map

m = Map(center=(47.51, 19), zoom=12)
for bus_stop in bus_stops:
    marker = Marker(location=bus_stop["location"], draggable=False)
    m.add_layer(marker);

m

A Jupyter Widget

De vajon a "9"-es busz az merre jár? Tudnál egy függvényt írni, amit ha meghívsz, bármilyen járathoz megkeresi a megállóinak a pozícióját?